In [1]:
import pandas as pd

wine = pd.read_csv('https://bit.ly/wine_csv_data')

data = wine[['alcohol', 'sugar', 'pH']].to_numpy()
target = wine['class'].to_numpy()

print(data.shape, target.shape)

(6497, 3) (6497,)


In [2]:
from sklearn.model_selection import train_test_split

train_input, test_input, train_target, test_target = train_test_split(data, target, test_size=0.2, random_state=42)

print(train_input.shape, test_input.shape)

(5197, 3) (1300, 3)


In [3]:
sub_input, val_input, sub_target, val_target = train_test_split(train_input, train_target, test_size=0.2, random_state=42)

print(sub_input.shape, val_input.shape)

(4157, 3) (1040, 3)


In [5]:
from sklearn.tree import DecisionTreeClassifier

dt = DecisionTreeClassifier(random_state = 42)
dt.fit(sub_input, sub_target)

print(dt.score(sub_input, sub_target))
print(dt.score(val_input, val_target))


0.9971133028626413
0.864423076923077


In [8]:
from sklearn.model_selection import cross_validate

scores = cross_validate(dt, train_input, train_target)
print(scores)
print(scores['test_score'])

{'fit_time': array([0.00845623, 0.00692558, 0.0072248 , 0.0072968 , 0.00680113]), 'score_time': array([0.00060916, 0.0005517 , 0.00056314, 0.00058579, 0.00051665]), 'test_score': array([0.86923077, 0.84615385, 0.87680462, 0.84889317, 0.83541867])}
[0.86923077 0.84615385 0.87680462 0.84889317 0.83541867]


In [9]:
import numpy as np

print(np.mean(scores['test_score']))

0.855300214703487


In [11]:
from sklearn.model_selection import StratifiedKFold

scores = cross_validate(dt, train_input, train_target, cv=StratifiedKFold(n_splits=10, shuffle=True, random_state=42))
print(np.mean(scores['test_score']))

0.8574181117533719


In [12]:
from sklearn.model_selection import GridSearchCV
params = {'min_impurity_decrease' : [0.0001, 0.0002, 0.0003, 0.0004, 0.0005]}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

GridSearchCV(cv=None, error_score=nan,
             estimator=DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=None,
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              presort='deprecated',
                                              random_state=42,
                                              splitter='best'),
             iid='deprecated', n_jobs=-1,
             param_grid={'min_impurity_decrease': [0.0001, 0.0002, 0.0003,
    

In [13]:
dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

0.9615162593804117
0.8653846153846154


In [15]:
print(gs.best_params_)

{'min_impurity_decrease': 0.0001}


In [16]:
gs.cv_results_

{'mean_fit_time': array([0.00912461, 0.00778732, 0.00842628, 0.00872965, 0.00619025]),
 'mean_score_time': array([0.00128331, 0.00070887, 0.00069771, 0.00126181, 0.00063238]),
 'mean_test_score': array([0.86819297, 0.86453617, 0.86492226, 0.86780891, 0.86761605]),
 'param_min_impurity_decrease': masked_array(data=[0.0001, 0.0002, 0.0003, 0.0004, 0.0005],
              mask=[False, False, False, False, False],
        fill_value='?',
             dtype=object),
 'params': [{'min_impurity_decrease': 0.0001},
  {'min_impurity_decrease': 0.0002},
  {'min_impurity_decrease': 0.0003},
  {'min_impurity_decrease': 0.0004},
  {'min_impurity_decrease': 0.0005}],
 'rank_test_score': array([1, 5, 4, 2, 3], dtype=int32),
 'split0_test_score': array([0.86923077, 0.87115385, 0.86923077, 0.86923077, 0.86538462]),
 'split1_test_score': array([0.86826923, 0.86346154, 0.85961538, 0.86346154, 0.86923077]),
 'split2_test_score': array([0.8825794 , 0.87680462, 0.87584216, 0.88161694, 0.8825794 ]),
 'split3_

In [17]:
print(gs.cv_results_['mean_test_score'])

best_index = np.argmax(gs.cv_results_['mean_test_score'])
print(gs.cv_results_['params'][best_index])

[0.86819297 0.86453617 0.86492226 0.86780891 0.86761605]
{'min_impurity_decrease': 0.0001}


In [20]:
params = {'min_impurity_decrease' : np.arange(0.0001, 0.001, 0.0001),
          'max_depth' : np.arange(5,20,1),
          'min_samples_split' : np.arange(2,100,10)}

gs = GridSearchCV(DecisionTreeClassifier(random_state=42), params, n_jobs=-1)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

print(gs.best_params_)

0.892053107562055
0.8615384615384616
{'max_depth': 14, 'min_impurity_decrease': 0.0004, 'min_samples_split': 12}


In [22]:
from scipy.stats import randint, uniform

rgen = randint(0,10)
np.unique(rgen.rvs(50000), return_counts=True)

(array([0, 1, 2, 3, 4, 5, 6, 7, 8, 9]),
 array([5014, 4894, 5100, 4967, 4936, 4884, 5061, 5094, 5081, 4969]))

In [23]:
ugen = uniform(0,1)
ugen.rvs(10)


array([0.34556625, 0.47812837, 0.96505398, 0.47212425, 0.91401008,
       0.85214783, 0.93778778, 0.57874812, 0.93480658, 0.79738521])

In [29]:
from sklearn.model_selection import RandomizedSearchCV

params = {'min_impurity_decrease' : uniform(0.0001, 0.001),
          'max_depth' : randint(5,20),
          'min_samples_split' : randint(2,100)}

gs = RandomizedSearchCV(DecisionTreeClassifier(random_state=42), params, n_iter=300, n_jobs=-1, random_state=42)
gs.fit(train_input, train_target)

dt = gs.best_estimator_
print(dt.score(train_input, train_target))
print(dt.score(test_input, test_target))

print(gs.best_params_)

0.8937848758899365
0.8630769230769231
{'max_depth': 16, 'min_impurity_decrease': 0.00037396112732113374, 'min_samples_split': 13}


In [30]:
print(np.max(gs.cv_results_['mean_test_score']))

0.8680019619456578
